<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Pre-trained-LM-selection-and-training/Experiments_Ktrain%2C_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorization 

Explicit stereotypes :
* Overt expression of social stereotypes (over generalized beliefs and expectancies of social categories)
* Crowdsourced using amazon mechanical turk.
  e.g. : "Asians are good in math"
* Datasets :
  1. Stereoset
  2. CrowsSpair

Implicit stereotypes :
  * Implicit or subtle projection of stereotypes as prejudiced attitude.
  * Often veiled or subtly projection of stereotypical behaviour and expectencies. 
  * Sometimes called "Micro-aggressions"- Unconsciously and sublty expresses prejudiced attitude.
  * Dataset:
    1. SocialBias Frames
    2. Microaggression (Not sure)

Datasets division :
  1. Stereoset
    * Categories :
        1. Profession - (827 + 810) -> 1637
        2. Race/Ethnicity - (242 + 962) -> 1204
        3. Gender - (242 + 255) -> 497
        4. Religion - (78 + 79) -> 157
    * Total : 2123 (Intersentence) + 2106 (Intrasentence) = 4229
  2. CrowsSpair
    * Categories :
      1. Race-color - 473 
      2. Gender/gender identity - 159
      3. Socioeconomic / occupation - 157
      4. Nationality - 148 
      5. Religion - 99
      6. Age - 73
      7. Sexual orientation - 72
      8. Disability - 57
      9. Physical appearance - 52
    * Total : 1290
  * Why?
    * Mostly explicit as they are crowdsourced.
    * For each target terms (Asian) from each domains (race,..); crowdworker writes sentence containing target term (Asians) with stereotype attribute; 
    * Both were compared which implies both datasets are of same type.

**Stats of Explicit stereo (Stereo + CrowSpair)** :
* Categories After combining:
    1. Ethnicity - 2559
    2. Profession - 1637
    3. Gender - 656
    4. Religion - 256
    5. Socio-economic / Occupation - 157
    6. Age - 73
    7. Sexual-orientation - 72
    8. Disability - 57
    9. Physical appearance - 52
* Total : 5519

Definitions **(TBD)**:

  1. Ethnicity :
  2. Profession :
  3. Gender :
  4. Religion :
  5. Occupaiton / Socioeconomic status :
  6. Age :
  7. Sexual Orientation :
  8. Disability :
  9. Physical Appearance :

Notes:

1. Combined race,race-color,nationality into ethnicity.
2. Combining Socioeconomic into profession category ?

Questions:

* Should I avoid Age, sexual orientation, disability and physical appearance categories as the samples are very few (`<100`)??

* Gender bias is explored by looking at associations between gender and occupation (GPT-3)?
  * Target names (he,she,..) and attributes(occupation 

* For inter-sentence samples, I am encdoing it as single sentence than multiple sentences.
  * Same `token_ids` for two sentences ??

* Should I combine socioeconomic/ocupation into occupation ??
  
  **Quora**:

  "An occupation is job.

  A profession is by definition employment where the needs of the     people you work for supercede your needs."

  A person proficient can do occupation in a different field or else both are same.

Experiments:

1. Train on stereoset and test on crowSpair
2. Train with only four categories 
  1. Ethnicity - 2559
  2. Profession - 1794
  3. Gender - 656
  4. Religion - 256
3. Combining explicit and Implicit i.e Stereo + CrowSpair + SocialBiasFrames

 * Categories:
  1. Ethnicity/Race/Nationality - 4660 + 2559
  2. Profession - 1794 
  3. Gender - 656 + 4023
  4. Religion - 256
  5. Culture - 2880
  6. Victim - 2231 ??
  7. Disabled/Dsiability - 854 + 57
  8. Social - 732 ??
  9. Body/Physical apperance - 508 + 52
 

## Implicit stereo 

Implicit stereotypes :
  * Social Bias frames
    * Categories :
      1. Race - 4660
      2. Gender - 4023
      3. Culture - 2880
      4. Victim - 2231
      5. Disabled - 854
      6. Social - 732
      7. Body - 508
    * Total : 15,888

Definitions (**TBD**) :

  1. Race :
  2. Gender :
  3. Culture :
  4. Victim :
  5. Disabled : 
  6. Social :
  7. Body :

Remarks :

* Too ambigious to put into a category ??

# Pytorch 


In [1]:
!pip install transformers==4.5.1 --quiet

     |████████████████████████████████| 2.1MB 29.1MB/s 
     |████████████████████████████████| 901kB 48.5MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 


In [2]:
!nvidia-smi

Sun May 16 16:19:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [5]:
df = pd.read_csv('/content/combined_stereo.csv',index_col=0)

In [6]:
df.head()

,bias_type,stereotype
0,race,Many people live in Ethiopia. The people are v...
1,race,My professor is a hispanic man. He came here i...
2,gender,The schoolgirl is walking down the street. She...
3,race,We'll likely fly over Cape Verde on our flight...
4,race,I know many people who are Russian. They love ...


In [7]:
df.bias_type.value_counts(normalize = True)*100

race          45.826437
profession    38.708915
gender        11.752187
religion       3.712462
Name: bias_type, dtype: float64

In [8]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

labels = df['bias_type']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)

LabelEncoder()

In [9]:
for cls in range(len(encoder.classes_)):
  print(cls,":" ,encoder.classes_[cls])

0 : gender
1 : profession
2 : race
3 : religion


In [10]:
encoded_labels = {}
for cls in range(len(encoder.classes_)):
  encoded_labels[encoder.classes_[cls]] = cls

In [11]:
dfLabelEncod = df

In [12]:
dfLabelEncod['Label_encoded'] = dfLabelEncod['bias_type'].replace(encoded_labels)

In [13]:
dfLabelEncod

,bias_type,stereotype,Label_encoded
0,race,Many people live in Ethiopia. The people are v...,2
1,race,My professor is a hispanic man. He came here i...,2
2,gender,The schoolgirl is walking down the street. She...,0
3,race,We'll likely fly over Cape Verde on our flight...,2
4,race,I know many people who are Russian. They love ...,2
...,...,...,...
4224,race,Arab people are associated with war.,2
4225,profession,He was a new chemist in our lab.,1
4226,gender,"The gentlemen stood around the fire, carefully...",0
4227,race,The common ukrainian is a supporter of a russi...,2


In [14]:
train_text, temp_text, train_labels, temp_labels = train_test_split(dfLabelEncod['stereotype'], dfLabelEncod['Label_encoded'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['bias_type'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [15]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [16]:
max_seq_len = 70

In [17]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [34]:
train_labels

3248    1
2782    0
3862    2
1554    1
3706    2
       ..
1015    2
1373    2
4189    1
1379    2
1160    2
Name: Label_encoded, Length: 2960, dtype: int64

In [35]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [36]:
train_y

tensor([1, 0, 2,  ..., 1, 2, 2])

In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [40]:
train_dataloader.dataset.tensors

(tensor([[  101,  1996, 10670,  ...,     0,     0,     0],
         [  101,  2026,  2767,  ...,     0,     0,     0],
         [  101,  2008,  2711,  ...,     0,     0,     0],
         ...,
         [  101,  2070,  2360,  ...,     0,     0,     0],
         [  101,  1996,  4496,  ...,     0,     0,     0],
         [  101,  1045,  2777,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 0, 2,  ..., 1, 2, 2]))

In [20]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [49]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,4)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [50]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [51]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [52]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[2.12643678 0.64572426 0.54572271 6.72727273]


In [53]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

In [54]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds


In [55]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [56]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 1.362
Validation Loss: 1.161

 Epoch 2 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 1.182
Validation Loss: 1.113

 Epoch 3 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 1.063
Validation Loss: 1.068

 Epoch 4 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.936
Validation Loss: 0.973

 Epoch 5 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.899
Validation Loss: 0.927

 Epoch 6 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.887
Validation Loss: 1.093

 Epoch 7 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.803
Validation Loss: 0.811

 Epoch 8 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.729
Validation Loss: 0.887

 Epoch 9 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.700
Validation Loss: 0.844

 Epoch 10 / 10
  Batch    50  of     93.

Evaluating...

Training Loss: 0.709
Validation L

In [57]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [58]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [59]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.68      0.72      0.70        75
           1       0.87      0.86      0.86       246
           2       0.94      0.92      0.93       291
           3       0.62      0.70      0.65        23

    accuracy                           0.86       635
   macro avg       0.78      0.80      0.79       635
weighted avg       0.87      0.86      0.87       635



# Ktrain 

## Experiment - 1 :

Train on stereoset and test on CrowSpair

Training dataset : combined_stereo (Inter and intrasentence stereoset)

Test dataset : CrowSpair

In [21]:
pip install ktrain

     |████████████████████████████████| 25.3MB 118kB/s 
     |████████████████████████████████| 6.8MB 19.4MB/s 
     |████████████████████████████████| 983kB 35.7MB/s 
     |████████████████████████████████| 266kB 36.0MB/s 
     |████████████████████████████████| 1.9MB 28.8MB/s 
     |████████████████████████████████| 1.2MB 30.1MB/s 
     |████████████████████████████████| 471kB 30.0MB/s 
     |████████████████████████████████| 3.3MB 28.6MB/s 
     |████████████████████████████████| 901kB 29.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=6446e9c41bb05b1d96d24b5aaf72e87b6ce00ee60239cf8c34251ee88595c815
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=b01fa24a4218ae80029b25bc04cd1aa79aaa437ae54db88ffb0f01c924cb2c7b
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [22]:
import ktrain
from ktrain import text
import pandas as pd

In [23]:
combined_stereo = pd.read_csv('/content/combined_stereo.csv', index_col = 0)

In [24]:
combined_stereo.bias_type.value_counts()

race          1938
profession    1637
gender         497
religion       157
Name: bias_type, dtype: int64

Undersampling :
* Race (200)
* Profession (200) 
* Gender (200)
* Religion (157)

Total : 757 samples 

In [75]:
race = combined_stereo.loc[(combined_stereo['bias_type']=='race')].sample(200)
profession = combined_stereo.loc[(combined_stereo['bias_type']=='profession')].sample(200)
gender = combined_stereo.loc[(combined_stereo['bias_type']=='gender')].sample(200)

In [76]:
religion = combined_stereo[combined_stereo['bias_type']=='religion']

In [77]:
combined_stereo_undersampled = pd.concat([race,profession,gender,religion])

In [79]:
combined_stereo_undersampled.bias_type.value_counts()

race          200
gender        200
profession    200
religion      157
Name: bias_type, dtype: int64

In [80]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0";

In [81]:
X = combined_stereo_undersampled.iloc[:,1].values
y = combined_stereo_undersampled.iloc[:,0].values

In [ ]:
print(X)

In [ ]:
print(y)

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [85]:
categories = combined_stereo_undersampled.bias_type.unique()
print(categories)

['race' 'profession' 'gender' 'religion']


In [86]:
model_name = 'distilbert-base-uncased'

trans = text.Transformer(model_name = model_name ,maxlen=512, class_names= categories)

In [87]:
train_df = trans.preprocess_train(X_train,y_train)
val_df = trans.preprocess_test(X_val,y_val)
test_df = trans.preprocess_train(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 22
	99percentile : 27


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 32


preprocessing train...
language: en
train sequence lengths:
	mean : 4
	95percentile : 17
	99percentile : 24


Is Multi-Label? False


In [88]:
model = trans.get_classifier()

In [89]:
learner = ktrain.get_learner(model,train_data = train_df,val_data= val_df, batch_size= 16)

In [90]:
# learner.lr_find(show_plot=True,max_epochs=3)

In [99]:
learner.fit_onecycle(2e-5,2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
38/38 [==============================] - 62s 2s/step - loss: 0.5362 - accuracy: 0.9702 - val_loss: 0.3122 - val_accuracy: 0.9737
Epoch 2/2
38/38 [==============================] - 62s 2s/step - loss: 0.1854 - accuracy: 0.9868 - val_loss: 0.1875 - val_accuracy: 0.9803


In [100]:
learner.validate(class_names=trans.get_classes())

              precision    recall  f1-score   support

      gender       0.98      0.98      0.98        47
  profession       0.97      0.97      0.97        31
        race       0.98      1.00      0.99        41
    religion       1.00      0.97      0.98        33

    accuracy                           0.98       152
   macro avg       0.98      0.98      0.98       152
weighted avg       0.98      0.98      0.98       152



array([[46,  1,  0,  0],
       [ 1, 30,  0,  0],
       [ 0,  0, 41,  0],
       [ 0,  0,  1, 32]])

In [101]:
predictor = ktrain.get_predictor(learner.model, preproc= trans)

Predictions on CrowSpair 

1. Race-color -> Race
2. Socioeconomic status -> Profession
3. Gender
4. Religion 

To Be done :

* Do label encoding for the four categories

In [102]:
predictions = predictor.predict(X_test)

In [103]:
len(predictions)

888

In [104]:
from sklearn.metrics import classification_report

In [105]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

      gender       0.98      0.75      0.85       159
  profession       0.23      1.00      0.37       157
        race       0.00      0.00      0.00       473
    religion       1.00      0.51      0.67        99

    accuracy                           0.37       888
   macro avg       0.55      0.56      0.47       888
weighted avg       0.33      0.37      0.29       888



### Validate using LIME visualization and Test on crowSpair 

In [ ]:
learner.view_top_losses(n=5, preproc=trans)

----------
id:295 | loss:1.3 | true:profession | pred:race)

----------
id:729 | loss:1.24 | true:profession | pred:race)

----------
id:629 | loss:1.02 | true:profession | pred:gender)

----------
id:709 | loss:0.99 | true:profession | pred:race)



In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)

In [ ]:
predictor.get_classes()

['gender', 'profession', 'race', 'religion']

In [ ]:
predictor.predict_proba(X_test[295])

array([0.01351515, 0.2735535 , 0.70263916, 0.01029227], dtype=float32)

In [ ]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
def get_probabilit_per_class(text):
  probabilit_per_class = dict(zip(predictor.get_classes(),predictor.predict_proba(text)*100))
  sort_orders = sorted(probabilit_per_class.items(), key=lambda x: x[1], reverse=True)
  return sort_orders

In [ ]:
predictor.explain(X_test[295])

In [ ]:
get_probabilit_per_class(X_test[295])

[('race', 70.263916),
 ('profession', 27.355349),
 ('gender', 1.3515147),
 ('religion', 1.0292267)]

In [ ]:
predictor.explain(X_test[729])

In [ ]:
get_probabilit_per_class(X_test[729])

[('race', 63.263546),
 ('profession', 29.076178),
 ('religion', 5.812295),
 ('gender', 1.8479778)]

In [ ]:
predictor.explain(X_test[629])

In [ ]:
get_probabilit_per_class(X_test[629])

[('gender', 61.906326),
 ('profession', 35.967545),
 ('race', 1.0874783),
 ('religion', 1.0386512)]

## Experiment - 2 :

Train with only four categories 
  1. Ethnicity - 2559
  2. Profession - 1794
  3. Gender - 656
  4. Religion - 256
  
Dataset : Explicitstereo 


## Experiment - 3

Combining explicit and Implicit i.e Stereo + CrowSpair + SocialBiasFrames

 * Categories:
  1. Ethnicity/Race/Nationality - 4660 + 2559
  2. Profession - 1794 
  3. Gender - 656 + 4023
  4. Religion - 256
  5. Culture - 2880
  6. Victim - 2231 ??
  7. Disabled/Dsiability - 854 + 57
  8. Social - 732 ??
  9. Body/Physical apperance - 508 + 52